# **Infrastructure as Code**

In [1]:
import boto3
import configparser
import json
import pandas as pd
import psycopg2

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# Access AWS
KEY = config.get('AWS','KEY') 
SECRET = config.get('AWS','SECRET') 

# DATA WARE HOUSE Confi
# Cluster
CLUSTER_TYPE = config.get('CLUSTER','CLUSTER_TYPE')
NUM_NODES = config.get('CLUSTER','NUM_NODES')
NODE_TYPE = config.get('CLUSTER','NODE_TYPE')

IAM_ROLE_NAME = config.get('CLUSTER','IAM_ROLE_NAME')
CLUSTER_IDENTIFIER = config.get('CLUSTER','CLUSTER_IDENTIFIER')

HOST = config.get('CLUSTER','host')
DB_NAME = config.get('CLUSTER','DB_NAME')
DB_USER = config.get('CLUSTER','DB_USER')
DB_PASSWORD = config.get('CLUSTER','DB_PASSWORD')
DB_PORT = config.get('CLUSTER','DB_PORT')

# IAM_ROLE 
ARN = config.get('IAM_ROLE','ARN')

# S3
LOG_DATA = config.get('S3','LOG_DATA')
LOG_JSONPATH = config.get('S3','LOG_JSONPATH')
SONG_DATA = config.get('S3','SONG_DATA')


In [3]:
# Clients for EC2, S3, IAM, RedShift
ec2 = boto3.resource('ec2',
                    region_name='us-west-2',
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                  region_name='us-west-2',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET)

iam = boto3.client('iam',
                  region_name='us-west-2',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',
                  region_name='us-west-2',
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET)

In [4]:
# Check Data Set on S3
print(SONG_DATA)
print(LOG_DATA)

's3://udacity-dend/song-data'
's3://udacity-dend/log-data'


In [6]:
source_bucket =  s3.Bucket('udacity-dend')


for obj in source_bucket.objects.filter(Prefix='song-data/A/A/A/TRAAAY'):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAYL128F4271A5B.json')


In [7]:
source_bucket =  s3.Bucket('udacity-dend')


for obj in source_bucket.objects.filter(Prefix='log-data/2018/11/2018-11-01'):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')


## Create IAM role for Reshift to access S3

In [31]:
# Create the IAM role for Redshift to access S3
try:
    print('Creating a new IAM Role')
    dwhRole = iam.create_role(Path='/',
                             RoleName=IAM_ROLE_NAME,
                             Description = "Allows Redshift clusters to call AWS services on your behalf.",
                             AssumeRolePolicyDocument=json.dumps(
                             {'Statement': [{'Action': 'sts:AssumeRole',
                                            'Effect': 'Allow',
                                            'Principal': {'Service': 'redshift.amazonaws.com'}}],
                             'Version':'2012-10-17'}))
    

except Exception as e:
    print(e)

Creating a new IAM Role


In [32]:
# Attach Policy to IAM Role
iam.attach_role_policy(RoleName = IAM_ROLE_NAME,
                      PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")['ResponseMetadata']['HTTPStatusCode']

200

In [33]:
# Get IAM role Arn
roleArn = iam.get_role(RoleName = IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

arn:aws:iam::475103112026:role/dwhRole


## Create Redshift Cluster

In [43]:
try:
    response = redshift.create_cluster(        
        # parameters for hardware
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        # parameters for identifiers & credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=DB_USER,
        MasterUserPassword=DB_PASSWORD,
        
        # parameter for role (to allow s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [45]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

,Key,Value
0,ClusterIdentifier,dwhcluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,dwhuser
4,DBName,dwh
5,Endpoint,"{'Address': 'dwhcluster.cmyptj7fsclo.us-west-2.redshift.amazonaws.com', 'Port': 5439}"
6,VpcId,vpc-8cef5bf4
7,NumberOfNodes,4


## Notes for ARN and DB ENDPOINT

In [46]:
ENDPOINT = myClusterProps['Endpoint']['Address']
ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", ENDPOINT)
print("DWH_ROLE_ARN :: ", ROLE_ARN)

DWH_ENDPOINT ::  dwhcluster.cmyptj7fsclo.us-west-2.redshift.amazonaws.com
DWH_ROLE_ARN ::  arn:aws:iam::475103112026:role/dwhRole


In [28]:
#### CAREFUL!!
#-- Uncomment & run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
#### CAREFUL!!

{'Cluster': {'ClusterIdentifier': 'dwhcluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'MasterUsername': 'dwhuser',
  'DBName': 'dwh',
  'Endpoint': {'Address': 'dwhcluster.cmyptj7fsclo.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2019, 6, 14, 5, 14, 49, 802000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-8cef5bf4',
  'AvailabilityZone': 'us-west-2c',
  'PreferredMaintenanceWindow': 'sat:09:30-sat:10:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'NumberOfNodes': 4,
  'PubliclyAccessible': True,
  'Encrypted': False,
  'Tags': [],
  'EnhancedVpcRouting': False,
  'IamRoles': [{'IamRoleArn': 

In [29]:
# Delete IAM role
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName = IAM_ROLE_NAME)

{'ResponseMetadata': {'RequestId': 'd8d8caed-8e68-11e9-b077-997927eb1b7f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd8d8caed-8e68-11e9-b077-997927eb1b7f',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Fri, 14 Jun 2019 05:54:17 GMT'},
  'RetryAttempts': 0}}

## RedShift Interface

In [56]:
try:
    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(HOST,DB_NAME,DB_USER,DB_PASSWORD,DB_PORT))
    cur = conn.cursor()
    conn.set_session(autocommit=True)
except psycopg2.Error as e: 
    print(e)

In [58]:
query = "SELECT * FROM stl_load_errors"
cur.execute(query)
row = cur.fetchall()
print(row)

[(100, 4, 100432, datetime.datetime(2019, 6, 15, 3, 42, 0, 87388), 12167, 91, 's3://udacity-dend/song-data/C/A/R/TRCARJQ128F425A389.json                                                                                                                                                                                                       ', 1, 'artist_name                                                                                                                    ', 'varchar   ', '256       ', 0, '{"song_id": "SOLAUEC12A8AE476BB", "num_songs": 1, "title": "Medication", "artist_name": "Spiritualized;Jason;Jason - Dulcimer/;Kate Radley - Vox continental/Farfisa/Tones/Drones/Tremeloes/;Sean Cook - Wha-monica/;Mark Refoy;Jon Mattock;Icon Hunt;Stuart Gordon;Balanescu Quartet;Rico;Rico - Tam Tam/;Bammie;Tim Sanders;Roddy Lorimar;Steve Sidwell;Chris Sharrack;Caroline Crawley;Marilyn McFarlane;Helen White", "artist_latitude": null, "year": 0, "duration": 498.72934, "artist_id": "ARVHQMD1269FB

**Roll UP** How many Songs is played in each month in 2018?

In [ ]:
query = """
SELECT COUNT(*)
FROM songPlay p JOIN dimTime t ON p.start_date = t.start_date
GROUP BY t.month
having t.year = 2018
LIMIT 10"""

cur.execute(query)
row = cur.fetchall()
print(row)

**Drill Down** How many song is played in 2018 July's each week?

In [ ]:
query = """
SELECT COUNT(*)
FROM songPlay p JOIN dimTime t ON p.start_date = t.start_date
GROUP BY t.week
HAVING t.year = 2018 AND t.month = 7
LIMIT 10"""
cur.execute(query)
row = cur.fetchall()
print(row)

**Slicing** How many femals are free users?

In [ ]:
query = """
SELECT COUNT(*)
FROM songPlay p JOIN dimUser u ON p.user_id = u.user_id
WHERE u.gender = 'M' AND u.level = 'free'
LIMIT 10"""
cur.execute(query)
row = cur.fetchall()
print(row)

**Dicing** Which song has been listened the most times during Summer(June - August)?

In [ ]:
query = """
SELECT COUNT(*)
FROM songPlay p JOIN dimTime t ON p.start_date = t.start_date
GROUP BY p.song_id
HAVING t.month in (6,7,8)
LIMIT 10"""
cur.execute(query)
row = cur.fetchall()
print(row)